## ETL V

In [1]:
import pandas as pd
import mysql.connector

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [2]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña


    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")


    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            


    def check_comunidades(self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        query_existe_comunidad = f"""
                SELECT DISTINCT comunidad FROM comunidades
                """
        mycursor.execute(query_existe_comunidad)
        comunidades = mycursor.fetchall()
        return comunidades
    

    def sacar_id_comunidad(self, comunidad):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idcomunidades FROM comunidades WHERE comunidad = '{comunidad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. "


In [3]:
tabla_comunidades = '''
CREATE TABLE IF NOT EXISTS `energía`.`comunidades` (
  `idcomunidades` INT NOT NULL AUTO_INCREMENT,
  `comunidad` VARCHAR(45),
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;
'''

In [4]:
tabla_nacional_renovable_no_renovable = """
CREATE TABLE IF NOT EXISTS `energía`.`nacional_renovable_no_renovable`(
`idnacional` INT NOT NULL AUTO_INCREMENT,
`porcentaje` DECIMAL,
`tipo_energia` VARCHAR(45),
`valor` DECIMAL,
`comunidades_idcomunidades` INT,
PRIMARY KEY (`idnacional`))
ENGINE = InnoDB;"""

In [5]:
tabla_comunidades_renovable_no_renovable = """
    CREATE TABLE IF NOT EXISTS `energía`.`comunidades_renovable_no_renovable`(
    `id` INT NOT NULL AUTO_INCREMENT,
    `porcentaje` DECIMAL,
    `tipo_energia` VARCHAR(45),
    `valor` DECIMAL,
    `idcomunidades` INT NOT NULL,
    PRIMARY KEY (`id`),
    CONSTRAINT `fk_comunidades_comunidades`
        FOREIGN KEY (`idcomunidades`)
        REFERENCES `energía`.`comunidades` (`idcomunidades`))
    ENGINE = InnoDB;"""

In [6]:
tabla_fechas = """
    CREATE TABLE IF NOT EXISTS `energía`.`fechas`(
    `idFechas` INT NOT NULL AUTO_INCREMENT,
    `fecha` DATE,
    `nacional_renovable_no_renovable_id` INT,
    `comunidades_renovable_no_renovable_id` INT,
    PRIMARY KEY (`idFechas`),
    CONSTRAINT `fk_nacional_fechas`
        FOREIGN KEY(`nacional_renovable_no_renovable_id`)
        REFERENCES `energía`.`nacional_renovable_no_renovable`(`idnacional`),
    CONSTRAINT `fk_comunidades_fechas`
        FOREIGN KEY (`comunidades_renovable_no_renovable_id`)
        REFERENCES `energía`.`comunidades_renovable_no_renovable` (`id`))
    ENGINE = InnoDB;"""

In [7]:
carga = Cargar("energía", "AlumnaAdalab")

In [8]:
carga.crear_bbdd()

Conexión realizada con éxito


In [9]:
carga.crear_insertar_tabla(tabla_comunidades)
carga.crear_insertar_tabla(tabla_nacional_renovable_no_renovable)
carga.crear_insertar_tabla(tabla_comunidades_renovable_no_renovable)
carga.crear_insertar_tabla(tabla_fechas)

In [10]:
carga.check_comunidades()

[]

In [11]:
df_comunidades = pd.read_csv('../datos/df_ccaa_clean.csv', index_col = 0)
df_comunidades.head(5)

,value,percentage,type,CCAA,fecha
0,10996921.24,0.28,renovable,Andalucía,2011-01-01
1,27608105.11,0.72,no renovable,Andalucía,2011-01-01
2,7005216.35,0.39,renovable,Aragón,2011-01-01
3,11119680.66,0.61,no renovable,Aragón,2011-01-01
4,338515.68,0.17,renovable,Cantabria,2011-01-01


In [12]:
for indice, fila in df_comunidades.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidades = f"""
                INSERT INTO comunidades (comunidad) 
                VALUES ("{fila["CCAA"]}");
                """  
    
    comunidades = carga.check_comunidades()
    list_comunidades = [i for tupla in comunidades for i in tupla]
    
    
    if fila['CCAA'] not in list_comunidades: 
        carga.crear_insertar_tabla(query_comunidades)

    else:
        print(f"{fila['CCAA']} ya esta en nuestra BBDD") 

Andalucía ya esta en nuestra BBDD
Aragón ya esta en nuestra BBDD
Cantabria ya esta en nuestra BBDD
Castilla - La Mancha ya esta en nuestra BBDD
Castilla y León ya esta en nuestra BBDD
Cataluña ya esta en nuestra BBDD
País Vasco ya esta en nuestra BBDD
Principado de Asturias ya esta en nuestra BBDD
Comunidad de Madrid ya esta en nuestra BBDD
Comunidad Foral de Navarra ya esta en nuestra BBDD
Comunitat Valenciana ya esta en nuestra BBDD
Extremadura ya esta en nuestra BBDD
Galicia ya esta en nuestra BBDD
Región de Murcia ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
Andalucía ya esta en nuestra BBDD
Andalucía ya esta en nuestra BBDD
Aragón ya esta en nuestra BBDD
Aragón ya esta en nuestra BBDD
Cantabria ya esta en nuestra BBDD
Cantabria ya esta en nuestra BBDD
Castilla - La Mancha ya esta en nuestra BBDD
Castilla - La Mancha ya esta en nuestra BBDD
Castilla y León ya esta en nuestra BBDD
Castilla y León ya esta en nuestra BBDD
Cataluña ya esta en nuestra BBDD
Cataluña ya esta e